In [1]:
import os
import pickle

from collections import Counter

# import pandas as pd
from sklearn.metrics import classification_report

import numpy as np
import torch
import torch.nn as nn

import transformers
from transformers import Trainer
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers.data.data_collator import DataCollatorWithPadding

import datasets
from datasets import Dataset
from datasets import ClassLabel
from datasets import load_metric

import evaluate # type: ignore

## Global variables

In [2]:
# DATA_FOLDER = '/notebooks/Data/bert_sequence_classification'
DATA_FILE = 'emotion_analysis_comics/bert/datasets/comics_dataset_complete.pt'
RESULTS_FOLDER = 'emotion_analysis_comics/bert/outputs'

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

device(type='cuda')

## Load data

In [5]:
dataset = torch.load(DATA_FILE)

/tmp/ipykernel_384097/2065342137.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dataset = torch.load(DATA_FILE)


In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['file_name', 'page_nr', 'panel_nr', 'balloon_nr', 'utterance', 'raw_annotation', 'raw_emotion', 'raw_speaker_id', 'emotion', 'speaker_id', 'split', 'utterance_emotion', 'unique_emotion'],
        num_rows: 5075
    })
    test: Dataset({
        features: ['file_name', 'page_nr', 'panel_nr', 'balloon_nr', 'utterance', 'raw_annotation', 'raw_emotion', 'raw_speaker_id', 'emotion', 'speaker_id', 'split', 'utterance_emotion', 'unique_emotion'],
        num_rows: 1097
    })
    validation: Dataset({
        features: ['file_name', 'page_nr', 'panel_nr', 'balloon_nr', 'utterance', 'raw_annotation', 'raw_emotion', 'raw_speaker_id', 'emotion', 'speaker_id', 'split', 'utterance_emotion', 'unique_emotion'],
        num_rows: 564
    })
})

In [7]:
l = dataset['test']['unique_emotion']

In [8]:
len(l)

1097

In [9]:
dataset['train']['utterance'][230]

"I… didn't expect this. How should I play it?"

In [10]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
label_names = list(set(dataset['train']['unique_emotion']))
label_nb = len(label_names)
labels = ClassLabel(num_classes=label_nb, names=label_names)

In [13]:
list_labels = dataset['test']['unique_emotion']

In [14]:
len(list_labels)

1097

In [15]:
counter = Counter(list_labels)

In [17]:
class_samples = []

for cl in labels.names:
    class_samples.append(counter[cl])

In [18]:
class_samples

[42, 212, 129, 21, 195, 321, 177]

In [19]:
# labels.num_classes

In [20]:
def tokenize(batch):
    tokens = tokenizer(batch['utterance'], truncation=True, padding=True, max_length=512)
    tokens['labels'] = labels.str2int(batch['unique_emotion'])
    return tokens

# this is just the text. if the results are nice, check transfer with text + topic 

In [21]:
dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/5075 [00:00<?, ? examples/s]

Map:   0%|          | 0/1097 [00:00<?, ? examples/s]

Map:   0%|          | 0/564 [00:00<?, ? examples/s]

In [22]:
dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

In [23]:
train_dataset = dataset['train'].shuffle(seed=42)
test_dataset = dataset['test'].shuffle(seed=42)
val_dataset = dataset['validation'].shuffle(seed=42)

In [24]:
dataset_d = {}
dataset_d['train'] = train_dataset
dataset_d['test'] = test_dataset
dataset_d['val'] = val_dataset

In [27]:
# sanity check
set(dataset_d['train']['split'])

{'TRAIN'}

In [28]:
# sanity check
set(dataset_d['val']['split'])

{'TRAIN'}

In [29]:
# sanity check
set(dataset_d['test']['split'])

{'TEST'}

In [30]:
# global variables
NUM_LABELS = label_nb
BATCH_SIZE = 256
NB_EPOCHS = 50

In [31]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=NUM_LABELS, device_map='cuda')
#model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
model.device

device(type='cuda', index=0)

In [33]:
class_counts = torch.tensor(class_samples, dtype=torch.float32).cuda()
class_weights = 1.0 / class_counts
class_weights = class_weights / class_weights.sum() * len(class_counts)

In [34]:
#class_weights.to(device)

In [35]:
# https://huggingface.co/transformers/main_classes/trainer.html
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss_fct = nn.CrossEntropyLoss(weight=class_weights)#(weight=class_weights)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [36]:
#metric = load_metric('f1', trust_remote_code=True)
metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    return metric.compute(predictions=predictions, references=labels, average='macro')

In [37]:
training_args = TrainingArguments(
    
    # output
    output_dir=RESULTS_FOLDER,          
    
    # params
    num_train_epochs=NB_EPOCHS,               # nb of epochs
    per_device_train_batch_size=BATCH_SIZE,   # batch size per device during training
    per_device_eval_batch_size=BATCH_SIZE,    # cf. paper Sun et al.
    learning_rate=1e-5,#2e-5,                 # cf. paper Sun et al.
#     warmup_steps=500,                         # number of warmup steps for learning rate scheduler
    warmup_ratio=0.1,                         # cf. paper Sun et al.
    weight_decay=0.01,                        # strength of weight decay
    
    # eval
    eval_strategy="steps",              # cf. paper Sun et al.
    eval_steps=20,                            # cf. paper Sun et al.
    
    # log
    logging_dir="emotion_analysis_comics/bert/logs",  
    logging_strategy='steps',
    logging_steps=20,
    
    # save
    save_strategy='steps',
    save_total_limit=1,
    # save_steps=20, # default 500
    load_best_model_at_end=True,              # cf. paper Sun et al.
    # metric_for_best_model='eval_loss' 
    metric_for_best_model='f1'
)

In [38]:
trainer = CustomTrainer( # Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

In [39]:
trainer.train()

Step,Training Loss,Validation Loss,F1
20,1.982300,1.980627,0.087122
40,1.955100,1.939983,0.093815
60,1.902700,1.901150,0.090283
80,1.841700,1.852060,0.082048
100,1.764900,1.805388,0.132563
120,1.678200,1.754731,0.214008
140,1.584100,1.698716,0.262166
160,1.497400,1.672707,0.272862
180,1.419000,1.641479,0.287892
200,1.335400,1.635361,0.298671


TrainOutput(global_step=1000, training_loss=0.7933314137458801, metrics={'train_runtime': 1867.7115, 'train_samples_per_second': 135.861, 'train_steps_per_second': 0.535, 'total_flos': 2.7906698234625e+16, 'train_loss': 0.7933314137458801, 'epoch': 50.0})

In [40]:
# save best model
#trainer.save_model(os.path.join("/notebooks/cascade_bert/saved_models", 'best-model-with-real-prev-probs'))

In [41]:
#model_file = os.path.join("/notebooks/cascade_bert/saved_models", 'best-model-with-real-prev-probs')

#model = BertForSequenceClassification.from_pretrained(model_file, num_labels=NUM_LABELS)
#model.to(device)
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [48]:
test_trainer = Trainer(model, data_collator=DataCollatorWithPadding(tokenizer))
test_raw_preds, test_labels, _ = test_trainer.predict(test_dataset)
test_preds = np.argmax(test_raw_preds, axis=1)

In [49]:
len(test_preds)

1097

In [50]:
test_labels

array([5, 6, 5, ..., 5, 5, 1])

In [51]:
test_preds

array([5, 6, 2, ..., 5, 1, 1])

In [46]:
# labels=['fear', 'anger', 'disgust', 'joy', 'sadness', 'surprise', 'neutral']

In [47]:
target_name = labels.int2str([0,1,2,3,4,5,6])
print(classification_report(test_labels, test_preds, target_names=target_name, digits=3)) # type: ignore

              precision    recall  f1-score   support

     neutral      0.152     0.286     0.198        42
        fear      0.456     0.340     0.389       212
     sadness      0.295     0.380     0.332       129
     disgust      0.000     0.000     0.000        21
         joy      0.444     0.405     0.424       195
       anger      0.525     0.533     0.529       321
    surprise      0.429     0.446     0.438       177

    accuracy                          0.421      1097
   macro avg      0.329     0.341     0.330      1097
weighted avg      0.430     0.421     0.422      1097

